# SCC0270 - Redes Neurais e Aprendizado Profundo
### Aula 8 - Prática (Redes neurais convolutivas)
**Daniel Penna Chaves Bertazzo - 10349561**

In [6]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical

# Item 1.1 - Preparação dos dados

###  Importação do *dataset* e concatenação em um único conjunto

In [7]:
# Carrega o dataset
# originalmente ja vem separado em treino e teste, mas aqui vamos juntar tudo
(data, target), (data_aux, target_aux) = tf.keras.datasets.mnist.load_data()

In [8]:
# Junta as instancias dos dois conjuntos em um unico dataset
data = np.concatenate((data, data_aux))

# Junta os targets dos dois conjuntos em um unico vetor
target = np.concatenate((target, target_aux))

# Para economizar memoria
del data_aux, target_aux

In [9]:
# Faz o one-hot enconding dos targets (compatibilidade com a saida de uma rede neural)
target = to_categorical(target)

In [10]:
print(data.shape, target.shape, sep='\n')

(70000, 28, 28)
(70000, 10)


### Definição das proporções dos cortes

In [16]:
# Calcula o tamanho de cada corte -> 10% até 100%
cuts = [int((x/10) * data.shape[0]) for x in range(1, 11)]

### Realização dos cortes

In [24]:
X = [] # Armazena os 10 conjuntos de instancias
y = [] # Armazena os 10 conjuntos de targets
for cut in cuts:
    X.append(data[:cut])
    y.append(target[:cut])

### Separação em treino e teste

In [40]:
from sklearn.model_selection import train_test_split

In [ ]:
X_10_train,  X_10_test,  y_10_train,  y_10_test  = train_test_split(X_10,  y_10,  test_size=0.3)
X_20_train,  X_20_test,  y_20_train,  y_20_test  = train_test_split(X_20,  y_20,  test_size=0.3)
X_30_train,  X_30_test,  y_30_train,  y_30_test  = train_test_split(X_30,  y_30,  test_size=0.3)
X_40_train,  X_40_test,  y_40_train,  y_40_test  = train_test_split(X_40,  y_40,  test_size=0.3)
X_50_train,  X_50_test,  y_50_train,  y_50_test  = train_test_split(X_50,  y_50,  test_size=0.3)
X_60_train,  X_60_test,  y_60_train,  y_60_test  = train_test_split(X_60,  y_60,  test_size=0.3)
X_70_train,  X_70_test,  y_70_train,  y_70_test  = train_test_split(X_70,  y_70,  test_size=0.3)
X_80_train,  X_80_test,  y_80_train,  y_80_test  = train_test_split(X_80,  y_80,  test_size=0.3)
X_90_train,  X_90_test,  y_90_train,  y_90_test  = train_test_split(X_90,  y_90,  test_size=0.3)
X_100_train, X_100_test, y_100_train, y_100_test = train_test_split(X_100, y_100, test_size=0.3)

### Ajustando as dimensões para ser compatível com o modelo do Keras

In [ ]:
X_10_train = X_10_train.reshape(X_10_train.shape[0], X_10_train.shape[1], X_10_train.shape[2], 1)
X_10_test  = X_10_test.reshape( X_10_test.shape[0],  X_10_test.shape[1],  X_10_test.shape[2],  1)

X_20_train = X_20_train.reshape(X_20_train.shape[0], X_20_train.shape[1], X_20_train.shape[2], 1)
X_20_test  = X_20_test.reshape( X_20_test.shape[0],  X_20_test.shape[1],  X_20_test.shape[2],  1)

X_30_train = X_30_train.reshape(X_30_train.shape[0], X_30_train.shape[1], X_30_train.shape[2], 1)
X_30_test  = X_30_test.reshape( X_30_test.shape[0],  X_30_test.shape[1],  X_30_test.shape[2],  1)

X_40_train = X_40_train.reshape(X_40_train.shape[0], X_40_train.shape[1], X_40_train.shape[2], 1)
X_40_test  = X_40_test.reshape( X_40_test.shape[0],  X_40_test.shape[1],  X_40_test.shape[2],  1)

X_50_train = X_50_train.reshape(X_50_train.shape[0], X_50_train.shape[1], X_50_train.shape[2], 1)
X_50_test  = X_50_test.reshape( X_50_test.shape[0],  X_50_test.shape[1],  X_50_test.shape[2],  1)

X_60_train = X_60_train.reshape(X_60_train.shape[0], X_60_train.shape[1], X_60_train.shape[2], 1)
X_60_test  = X_60_test.reshape( X_60_test.shape[0],  X_60_test.shape[1],  X_60_test.shape[2],  1)

X_70_train = X_70_train.reshape(X_70_train.shape[0], X_70_train.shape[1], X_70_train.shape[2], 1)
X_70_test  = X_70_test.reshape( X_70_test.shape[0],  X_70_test.shape[1],  X_70_test.shape[2],  1)

X_80_train = X_80_train.reshape(X_80_train.shape[0], X_80_train.shape[1], X_80_train.shape[2], 1)
X_80_test  = X_80_test.reshape( X_80_test.shape[0],  X_80_test.shape[1],  X_80_test.shape[2],  1)

X_90_train = X_90_train.reshape(X_90_train.shape[0], X_90_train.shape[1], X_90_train.shape[2], 1)
X_90_test  = X_90_test.reshape( X_90_test.shape[0],  X_90_test.shape[1],  X_90_test.shape[2],  1)

X_100_train = X_100_train.reshape(X_100_train.shape[0], X_100_train.shape[1], X_100_train.shape[2], 1)
X_100_test  = X_100_test.reshape( X_100_test.shape[0],  X_100_test.shape[1],  X_100_test.shape[2],  1)

#  Item 1.2 - Criação do modelo e implementação das redes
A arquitetura utilizada para todos os cortes será a seguinte:
* **Primeira camada:** convolução com 32 filtros 3x3 e ativação relu + max_pooling 2x2

* **Segunda camada:** convolução com 64 filtros 3x3 e ativação relu + max_pooling 2x2

* **Última camada:** *fully connected* com 10 neurônios (número de classes) e ativação softmax

In [ ]:
X_10_train.shape

In [ ]:
model_10 = Sequential()

model_10.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=X_10_train.shape[1:]))